In [ ]:
!pip install gluoncv

In [1]:
import boto3
from IPython.display import clear_output, Image, display, HTML
import numpy as np
import cv2
import base64
from bokeh.plotting import figure
from bokeh.io import output_notebook, show, push_notebook
import time
import json
output_notebook()

Loading BokehJS ...

In [2]:
STREAM_NAME = "pi4-001"
kvs = boto3.client("kinesisvideo")
# Grab the endpoint from GetDataEndpoint
endpoint = kvs.get_data_endpoint(
    APIName="GET_HLS_STREAMING_SESSION_URL",
    StreamName=STREAM_NAME
)['DataEndpoint']
print(endpoint)

https://b-647daf39.kinesisvideo.us-west-2.amazonaws.com


In [3]:
class VideoPlayer(object):
    def __init__(self):
        self._init = False
        self._myImage = None
        
    def __call__(self, frame):
        if frame is None:
            return
        if self._init is False:
            self.init_display(frame)
            self._init = True
        else:
            self.update_display(frame)

    def init_display(self, frame):
        assert frame is not None
        frame=cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA) # because Bokeh expects a RGBA image
#         frame=cv2.flip(frame, -1) # because Bokeh flips vertically
        frame=cv2.flip(frame, 0) # because Bokeh flips vertically
        width=frame.shape[1]
        height=frame.shape[0]
        p = figure(x_range=(0,width), y_range=(0,height), output_backend="webgl", width=width, height=height)
        self._myImage = p.image_rgba(image=[frame], x=0, y=0, dw=width, dh=height)
        show(p, notebook_handle=True)
    
    def update_display(self, frame):
        assert frame is not None
        frame=cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
#         frame=cv2.flip(frame, -1)
        frame=cv2.flip(frame, 0) 
        self._myImage.data_source.data['image']=[frame]
        push_notebook()

In [4]:
# import sagemaker
# sagemaker_endpoint_name = 'object-detection-2019-10-18-17-56-41-925'
# sagemaker_endpoint = sagemaker.predictor.RealTimePredictor(sagemaker_endpoint_name)
# sagemaker_endpoint.content_type = 'image/jpeg'
from object_detection import ObjectDetection
objectDetection = ObjectDetection()

In [5]:
def detection_result_process(frame, classes, class_IDs, scores, bounding_boxes, hand_cnt, no_hand_cnt, start_trans, in_trans, curr_item_cnt, max_item_cnt):
    thres = 0.5
    if class_IDs is not None and len(class_IDs) == 1:
            if len(class_IDs[0]) >= 1:
                hand = False
                item_cnt = [0, 0, 0, 0]
                for i in range(len(class_IDs[0])):
                    if scores[0][i] > thres:
                        class_ID = int(class_IDs[0][i])
                        score = float(scores[0][i])
                        bounding_box = bounding_boxes[0][i]
                        # print('class_ID:', class_ID, 'score:', score, 'bounding_box:', bounding_box)
                        xmin, ymin, xmax, ymax = [int(x) for x in bounding_box]
                        cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), color=(0, 255, 0), thickness=1)
                        cv2.putText(frame, str(classes[class_ID])+':'+str(round(score, 2)), (xmin+10, ymin+10), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, thickness=1, color=(255, 255, 255))
                        
                        if class_ID == 4:
                            hand_cnt += 1
                            no_hand_cnt = 0
                            hand = True
                        else:
                            item_cnt[class_ID] += 1
                        
                for i in range(len(item_cnt)):
                    item_cnt[i] = min(item_cnt[i], max_item_cnt[i])
                        
                if not hand:
                    no_hand_cnt += 1
                    
                if hand_cnt > 3:
                    start_trans = True
                    in_trans = True
                        
                if no_hand_cnt > 3:
                    start_trans = False
                    
                if start_trans:
                    msg = 'Start Transaction'
                else:
                    msg = 'End Transaction'
                    if in_trans:
                        for i in range(len(item_cnt)):
                            change_item = item_cnt[i]-curr_item_cnt[i]
                            if change_item != 0:
                                msg += ' '+classes[i]+': '+str(change_item)
                        in_trans = False
                        curr_item_cnt = item_cnt
                    else:
                        for i in range(len(item_cnt)):
                            curr_item_cnt[i] = max(curr_item_cnt[i], item_cnt[i])
                msg2 = ''
                for i in range(len(curr_item_cnt)):
                    msg2 += classes[i]+': '+str(curr_item_cnt[i])+' '
                
                cv2.putText(frame, msg, (40, 40), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.8, thickness=2, color=(255, 255, 255))
                cv2.putText(frame, msg2, (40, 60), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, thickness=1, color=(255, 255, 255))
    return frame, hand_cnt, no_hand_cnt, start_trans, in_trans, curr_item_cnt, msg, msg2

In [7]:
# Grab the HLS Stream URL from the endpoint
kvam = boto3.client("kinesis-video-archived-media", endpoint_url=endpoint)
url = kvam.get_hls_streaming_session_url(
    StreamName=STREAM_NAME,
    PlaybackMode="LIVE"
)['HLSStreamingSessionURL']
vcap = cv2.VideoCapture(url)
player = VideoPlayer()

hand_cnt = 0
no_hand_cnt = 0
start_trans = False
in_trans = False
curr_item_cnt = [0, 0, 0, 0]
max_item_cnt = [2, 2, 1, 1]

while(True):
    # Capture frame-by-frame
    read_start = time.time()
    ret, frame = vcap.read()
    read_end = time.time()
    #print('read time:', read_end-read_start)

    if frame is not None:
        start = time.time()
        frame = cv2.flip(frame, -1)
        
        # save image
        #filename = 'frame_'+str(time.time())+'.jpg'
        #cv2.imwrite(filename, frame)

        # use SageMaker
        #with open(filename, 'rb') as image:
        #    f = image.read()
        #    b = bytearray(f)
        #result = sagemaker_endpoint.predict(b)
        #print(json.loads(result))
        
        # use GluonCV
        detect_start = time.time()
        class_IDs, scores, bounding_boxes = objectDetection.detect_image(frame)
        #print('class_IDs:', class_IDs)
        #print('scores:', scores)
        #print('bounding_boxes:', bounding_boxes)
        detect_end = time.time()
        #print('detect time:', detect_end-detect_start)
        
        frame, hand_cnt, no_hand_cnt, start_trans, in_trans, curr_item_cnt, msg, msg2 = detection_result_process(frame, objectDetection.classes, class_IDs, scores, bounding_boxes, hand_cnt, no_hand_cnt, start_trans, in_trans, curr_item_cnt, max_item_cnt)
        #print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(detect_end)), 'msg:', msg)
        #print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(detect_end)), 'msg2:', msg2)
        
        # Display the resulting frame
        #cv2.imwrite(filename, frame)
        player(frame)
        end = time.time()
        #print('all time:', end-start)
        
        #t = 1
        #time.sleep(t)
        #vcap.set(cv2.CAP_PROP_POS_FRAMES, t)
    
        #vcap.set(cv2.CAP_PROP_POS_FRAMES, end-start)
#         vcap.set(cv2.CAP_PROP_POS_FRAMES, 0.05)

        #break

        # Press q to close the video windows before it ends if you want
        #if cv2.waitKey(22) & 0xFF == ord('q'):
        #    break
    else:
        print("Frame is None")
        break

# When everything done, release the capture
vcap.release()
print("Video stop")

KeyboardInterrupt: 